In [9]:
import sqlite3
import os
import openai
import time
import datetime
import chromadb

openai.api_key = os.getenv("OPENAI_API_KEY")
SQLITE_DB_NAME = "../data/reviews_database.sqlite"

In [ ]:
# create the DB table which we're going to used for the compressed reviews
conn = sqlite3.connect(SQLITE_DB_NAME)
cursor = conn.cursor()

cursor.execute(
            """
            CREATE TABLE IF NOT EXISTS compressed_content (
                reviewId INTEGER PRIMARY KEY,
                complete_content TEXT NOT NULL,
                vibes TEXT NOT NULL,
                genres TEXT NOT NULL,
                instruments TEXT NOT NULL
            )
            """
        )

conn.close()


In [16]:
# call ChatGPT and ask it to extract descriptive content from the review
def get_compressed_review(review):
  system_prompt = """You will be given an album review. Read the review and extract any words or phrases that describe the music itself or its emotions/vibes.
  
  Your response should be formatted like this:
  
  INSTRUMENTS: <description of instruments
  GENRES: <list of genres
  EMOTIONS/VIBES: <description of emotions/vibes>
  Do not use full sentences or punctuation."""

  response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
          {"role": "system", "content": system_prompt},
          {"role": "user", "content": review},
      ],
    temperature=0.6,
    max_tokens=256
  )

  return response.choices[0].message.content

# parse the compressed review into the correct sections and return it as a dictionary
def parse_compressed_review(review):
  lines = review.strip().split('\n')
  parsed_review = {}
  for line in lines:
    section, content = line.split(':', 1)
    parsed_review[section.strip().lower()] = content.strip()
  
  return parsed_review



In [2]:
conn = sqlite3.connect(SQLITE_DB_NAME)
cursor = conn.cursor()

# make a set of reviewIds that have already been processed so we can easily skip them if
# the script fails part way through and needs to be restarted. it's a bit hacky but ¯\_(ツ)_/¯
cursor.execute("SELECT reviewId FROM compressed_content")
review_id_rows = cursor.fetchall()
review_id_set = set(row[0] for row in review_id_rows)

cursor.execute("SELECT reviewId, content FROM content")
rows = cursor.fetchall()

for row in rows[0:3]:
    print(row)


cursor.execute("SELECT * FROM compressed_content")
rows = cursor.fetchall()

for row in rows[0:3]:
    print(row)

conn.close()

OperationalError: no such table: compressed_content

In [18]:
conn = sqlite3.connect(SQLITE_DB_NAME)
cursor = conn.cursor()

# make a set of reviewIds that have already been processed so we can easily skip them if
# the script fails part way through and needs to be restarted. it's a bit hacky but ¯\_(ツ)_/¯
cursor.execute("SELECT reviewId FROM compressed_content")
review_id_rows = cursor.fetchall()
review_id_set = set(row[0] for row in review_id_rows)

cursor.execute("SELECT reviewId, content FROM content")
rows = cursor.fetchall()

start_time = time.time()

# get/parse compressed reviews for each review
for i, row in enumerate(rows):
    review_id, review = row
    if review_id in review_id_set:
        print(f"Skipping reviewId {review_id}")
        continue

    elapsed_time = time.time() - start_time
    elapsed_time_str = str(datetime.timedelta(seconds=elapsed_time))
    print(f"Index {i}/{len(rows)}. Elapsed time: {elapsed_time_str}.")

    try:
        # have GPT compress the review
        print(f"Compressing reviewId {review_id} with content {review}")
        compressed_review = get_compressed_review(review)
        print(f"Finished compressing reviewId {review_id}: {compressed_review}")

        # parse it
        parsed_review = parse_compressed_review(compressed_review)
        print(f"Finished parsing reviewId {review_id}: {parsed_review}")
        vibes = parsed_review["emotions/vibes"]
        genres = parsed_review["genres"]
        instruments = parsed_review["instruments"]
        complete_content = f"{genres}, {vibes}, {instruments}"

        # store it
        cursor.execute("""INSERT INTO compressed_content
            (reviewId, complete_content, vibes, genres, instruments)
            VALUES (?, ?, ?, ?, ?)""", (review_id, complete_content, vibes, genres, instruments))
        conn.commit()
    except Exception as e:
        print(f"Error while processing reviewId {review_id}: {e}")
        continue

conn.close()


Skipping reviewId 22703
Skipping reviewId 22721
Skipping reviewId 22659
Skipping reviewId 22661
Skipping reviewId 22725
Skipping reviewId 22722
Skipping reviewId 22704
Skipping reviewId 22694
Skipping reviewId 22714
Skipping reviewId 22724
Skipping reviewId 22715
Skipping reviewId 22745
Skipping reviewId 22700
Skipping reviewId 22720
Skipping reviewId 22699
Skipping reviewId 22665
Skipping reviewId 22666
Skipping reviewId 22719
Skipping reviewId 22667
Skipping reviewId 22691
Skipping reviewId 22702
Skipping reviewId 22718
Skipping reviewId 22706
Skipping reviewId 22695
Skipping reviewId 22713
Skipping reviewId 22705
Skipping reviewId 22532
Skipping reviewId 22701
Skipping reviewId 22707
Skipping reviewId 22708
Skipping reviewId 22559
Skipping reviewId 22692
Skipping reviewId 22677
Skipping reviewId 22469
Skipping reviewId 22711
Skipping reviewId 22678
Skipping reviewId 22664
Skipping reviewId 22685
Skipping reviewId 22690
Skipping reviewId 22689
Skipping reviewId 22697
Skipping reviewI

KeyboardInterrupt: 

In [ ]:
# make embeddings of the concatenated version of compressed reviews and save in Chroma
conn = sqlite3.connect(SQLITE_DB_NAME)
cursor = conn.cursor()

# load concatenated review text
cursor.execute("SELECT reviewId FROM compressed_content")
rows = cursor.fetchall()

documents = []
ids = []
metadatas = []


# pass into Chroma - with persist option
# wrap in for loop - chunks
    # persist at end of loop
openai_ef = embedding_functions.OpenAIEmbeddingFunction(
    api_key=OPENAI_API_KEY,
    model_name=EMBEDDING_MODEL)

client = chromadb.Client(
    Settings(chroma_db_impl="duckdb+parquet",
             persist_directory="embeddings_data"))

collection = client.get_or_create_collection(name="reviews", embedding_function=openai_ef)